In [ ]:
import arviz as az
import xarray

i = 7
bn = 'viet_7'
az.rcParams['data.load'] = 'eager'
idata: az.InferenceData = az.from_netcdf(f'{bn}.nc')

idata.posterior['A'].isel(harmonic=0).median(dim=['chain', 'draw']).plot()
# load xarray from netcdf

y = xarray.open_dataarray(f'y_{bn}.nc')
X = xarray.open_dataarray(f'X_{bn}.nc')

In [ ]:
idata.posterior['linear_effect'].isel(harmonic=0).median(dim=['chain', 'draw']).plot()

In [ ]:
az.plot_posterior(data=idata, var_names=['a_mu'], coords={'harmonic': 0})

In [ ]:
# Plot posterior  A over last year
print(idata.posterior['A'].sel(harmonic=0, year=17).median(dim=['chain', 'draw']))

#az.plot_posterior(data=idata, var_names=['A'], coords={'harmonic': 0, 'year': 17}, point_estimate='median')

import matplotlib.pyplot as plt

idata.posterior['A'].sel(harmonic=0, year=17, location='BinhDinh').plot.hist()
idata.posterior['a_mu'].sel(harmonic=0, location='BinhDinh').plot.hist()

In [ ]:
coords = y.coords
max_month = idata.posterior['last_mu'].max(dim=('month'))
for location in coords['location']:
    max_month.sel(location=location, year=17).plot.hist(density=True)
    max_month.sel(location=location).plot.hist(density=True)
    plt.show()

In [ ]:
print(y.sel(year=17).values)


In [ ]:
import numpy as np
import pymc as pm

from chap_pymc.curve_parametrizations.fourier_parametrization import (
    FourierParametrization,
)

harmonic_ = np.arange(idata.posterior.dims['harmonic'])
print(harmonic_)
# Convert xarray coords to dict with values
coords = {k: v.values for k, v in y.coords.items()} | {k: v.values for k, v in X.coords.items()} | {'harmonic': harmonic_}
print(coords)
with pm.Model(coords=coords) as model:
    FourierParametrization().get_regression_model(X, y)

    new_a_samples = pm.sample_posterior_predictive(idata, var_names=['y_obs', 'A'], random_seed=42)


In [ ]:
new_a_samples.posterior_predictive['y_obs'].sel(location='BinhDinh').median(dim=['chain', 'draw']).plot()

In [ ]:
new_a_samples.posterior_predictive['A'].sel(harmonic=0, location='BinhDinh').mean(dim=['chain', 'draw']).plot()

In [ ]:
for location in y.location.values:
    plt.figure()
    new_a_samples.posterior_predictive['y_obs'].sel(location=location).mean(dim=['chain', 'draw', 'month']).plot()
    y.sel(location=location).mean(dim='month').plot()
    plt.title(location)